In [1]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook
import pickle
import os
import logging
import time
from IPython.core.debugger import set_trace
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F

from utils.utils import save_checkpoint, load_checkpoint, set_logger
from utils.gpu_utils import set_n_get_device

from dataset.dataset import prepare_trainset

#from model.model_unet_classify_zero import UNetResNet34
#from model.model_unet_classify_zero2 import UNetResNet34
from model.efficientnet_pytorch.efficientnet_pytorch import EfficientNet

from model.deeplab_model_kaggler.lr_scheduler import LR_Scheduler

%matplotlib inline

In [2]:
######### Config the training process #########
#device = set_n_get_device("0, 1, 2, 3", data_device_id="cuda:0")#0, 1, 2, 3, IMPORTANT: data_device_id is set to free gpu for storing the model, e.g."cuda:1"
MODEL = 'efficientnet-b5' #'b3','b5'
print('====MODEL ACHITECTURE: %s===='%MODEL)

device = set_n_get_device("0", data_device_id="cuda:0")#0, 1, 2, 3, IMPORTANT: data_device_id is set to free gpu for storing the model, e.g."cuda:1"
multi_gpu = None #[0, 1] #None

SEED = 2008
debug = False# if True, load 100 samples
IMG_SIZE = (256,1600)
BATCH_SIZE = 8
NUM_WORKERS = 24
torch.cuda.manual_seed_all(SEED)

====MODEL ACHITECTURE: efficientnet-b5====


## the dataset

In [3]:
train_dl, val_dl = prepare_trainset(BATCH_SIZE, NUM_WORKERS, SEED, IMG_SIZE, debug)

Count images in train/test folder:  5546 3698
Count of trainset (for training):  4714
Count of validset (for training):  832


In [4]:
for i, (images, masks) in enumerate(train_dl):
    images = images.to(device=device, dtype=torch.float)
    #1 for non-zero-mask
    truth = (torch.sum(masks.reshape(masks.size()[0], masks.size()[1], -1), dim=2, keepdim=False)!=0).to(device=device, dtype=torch.float)
    if i==0:
        break

In [5]:
images.size(), truth.size()

(torch.Size([8, 1, 256, 1600]), torch.Size([8, 4]))

In [6]:
#truth

## the model

In [6]:
net = EfficientNet.from_name(MODEL, debug=debug)#override_params={'num_classes': 1}
net.load_state_dict(torch.load(glob.glob('../model/%s*'%MODEL)[0]))
in_features = net._fc.in_features
net._fc = nn.Linear(in_features, 4)#num_classes=1
net = net.cuda(device=device)
print(glob.glob('../model/%s*'%MODEL)[0])
print('====Loading pretrained weights done====')

if multi_gpu is not None:
    net = nn.DataParallel(net, device_ids=multi_gpu)

../model/efficientnet-b5-586e6cc6.pth
====Loading pretrained weights done====


In [7]:
logit = net(images)

In [8]:
logit.size()

torch.Size([8, 4])

In [19]:
# Loss_FUNC = nn.MultiLabelMarginLoss()
# loss = Loss_FUNC(torch.sigmoid(logit), truth.long())
# loss

tensor(1.9995, device='cuda:0', grad_fn=<MultilabelMarginLossBackward>)

In [8]:
if multi_gpu:
    loss = net.module.criterion(logit, truth)
else:
    loss = net.criterion(logit, truth)

loss

tensor(0.7063, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [9]:
if multi_gpu:
    _, metric, tn, fp, fn, tp, pos_percent = net.module.metric(logit, truth)
else:
    _, metric, tn, fp, fn, tp, pos_percent = net.metric(logit, truth)

metric, tn, fp, fn, tp, pos_percent

(0.4037, 11, 16, 3, 2, 0.5625)

In [ ]:
%%time
#label_df = pd.read_csv('data/raw/NIH/NIH external data/label_df_pneumothorax.csv').set_index('img_id')

for idx, img_id in enumerate(label_df.index):
    if idx>10000:
        break
    has_pneumothorax = label_df.loc[img_id, 'has_pneumothorax']


In [ ]:
for idx, f in enumerate(glob.glob('data/raw/NIH/NIH external data/images*/*')):
    img = plt.imread(f)
    if len(img.shape)==3:
        print(img.shape)
        break
    if idx>100:
        break
#img_path = glob.glob('data/raw/NIH/NIH external data/images*/00000001_000.png')[0]#00000003_000.png
#img = plt.imread(img_path)

In [ ]:
img.shape

In [ ]:
#plt.imshow(img)

## predict the validset, and analyse  
**first, predict zero-nonzero-mask**

In [ ]:
#move checkpoint from gamma machine to here
cd checkpoint
scp -r endi.niu@10.171.36.214:/home/endi.niu/SIIM/checkpoint/nonzero_classifier_UNetResNet34_768_v1_seed1234/ nonzero_classifier_UNetResNet34_768_v1_seed1234
cd logging
scp -r endi.niu@10.171.36.214:/home/endi.niu/SIIM/logging/nonzero_classifier_UNetResNet34_768_v1_seed1234.log nonzero_classifier_UNetResNet34_768_v1_seed1234.log

In [1]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook
import pickle
import os
import logging
import time
from IPython.core.debugger import set_trace

import torch
import torch.nn as nn
import torch.nn.functional as F

from utils.utils import save_checkpoint, load_checkpoint, set_logger
from utils.gpu_utils import set_n_get_device

#from model.model_unet_classify_zero import UNetResNet34, predict_proba
from model.efficientnet_pytorch.efficientnet_pytorch.model import EfficientNet, predict_proba

from dataset.dataset import prepare_trainset

from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

%matplotlib inline

In [2]:
######### Config the training process #########
#device = set_n_get_device("0, 1, 2, 3", data_device_id="cuda:0")#0, 1, 2, 3, IMPORTANT: data_device_id is set to free gpu for storing the model, e.g."cuda:1"
MODEL = 'efficientnet-b5' #'b3','b5'
#AUX_LOGITS = True#False, only for 'INCEPTION_V3'
print('====MODEL ACHITECTURE: %s===='%MODEL)

device = set_n_get_device("0,1", data_device_id="cuda:0")#0, 1, 2, 3, IMPORTANT: data_device_id is set to free gpu for storing the model, e.g."cuda:1"
multi_gpu = [0, 1]#use 2 gpus

SEED = 2024
debug = False # if True, load 100 samples
IMG_SIZE = (512, 768)
BATCH_SIZE = 32
NUM_WORKERS = 24

====MODEL ACHITECTURE: efficientnet-b5====


In [3]:
train_dl, val_dl = prepare_trainset(BATCH_SIZE, NUM_WORKERS, SEED, IMG_SIZE, debug)

Count images in train/test folder:  5546 3698
Count of trainset (for training):  4714
Count of validset (for training):  832


In [4]:
# y should be makeup
y_valid = []
for i, (images, masks) in enumerate(val_dl):
    #if i==10:
    #    break
    #truth = masks.to(device=device, dtype=torch.float)
    truth = (torch.sum(masks.reshape(masks.size()[0], masks.size()[1], -1), dim=2, keepdim=False)!=0)#.to(device=device, dtype=torch.float)
    y_valid.append(truth.numpy())
y_valid = np.concatenate(y_valid, axis=0)
y_valid.shape

(832, 4)

In [5]:
y_valid.mean()

0.5336538461538461

In [12]:
%%time

ensemble = True

#### multiple fold ensemble sigmoids
if ensemble:
    checkpoint_path_list = ['../checkpoint/nonzero_classifier_efficientnet-b5_512x768_v4_seed2025/best.pth.tar', 
                            '../checkpoint/nonzero_classifier_efficientnet-b5_512x768_v4_seed2026/best.pth.tar', 
                            '../checkpoint/nonzero_classifier_efficientnet-b5_512x768_v4_seed2027/best.pth.tar', 
                            '../checkpoint/nonzero_classifier_efficientnet-b5_512x768_v4_seed2028/best.pth.tar', 
                            '../checkpoint/nonzero_classifier_efficientnet-b5_512x768_v4_seed2029/best.pth.tar']
    
    preds_valid = None
    for checkpoint_path in checkpoint_path_list:
        net = EfficientNet.from_name(MODEL, debug=debug)#override_params={'num_classes': 1}
        in_features = net._fc.in_features
        net._fc = nn.Linear(in_features, 4)#num_classes=1
        net = net.cuda(device=device)
        
        net, _ = load_checkpoint(checkpoint_path, net)
        
        if multi_gpu is not None:
            net = nn.DataParallel(net, device_ids=multi_gpu)

        _preds_valid = predict_proba(net, val_dl, device, multi_gpu=multi_gpu, mode='valid', tta=True)
        if preds_valid is None:
            preds_valid = _preds_valid
        else:
            preds_valid += _preds_valid
        preds_valid /= len(checkpoint_path_list)
        print('Complete ', checkpoint_path)

#### single fold
else:
    checkpoint_path = '../checkpoint/nonzero_classifier_efficientnet-b5_512x768_v4_seed2025/best.pth.tar'

    net = EfficientNet.from_name(MODEL, debug=debug)#override_params={'num_classes': 1}
    in_features = net._fc.in_features
    net._fc = nn.Linear(in_features, 4)#num_classes=1
    net = net.cuda(device=device)

    net, _ = load_checkpoint(checkpoint_path, net)

    if multi_gpu is not None:
        net = nn.DataParallel(net, device_ids=multi_gpu)

    preds_valid = predict_proba(net, val_dl, device, multi_gpu=multi_gpu, mode='valid', tta=True)


use TTA
Complete  ../checkpoint/nonzero_classifier_efficientnet-b5_512x768_v4_seed2025/best.pth.tar
use TTA
Complete  ../checkpoint/nonzero_classifier_efficientnet-b5_512x768_v4_seed2026/best.pth.tar
use TTA
Complete  ../checkpoint/nonzero_classifier_efficientnet-b5_512x768_v4_seed2027/best.pth.tar
use TTA
Complete  ../checkpoint/nonzero_classifier_efficientnet-b5_512x768_v4_seed2028/best.pth.tar
use TTA
Complete  ../checkpoint/nonzero_classifier_efficientnet-b5_512x768_v4_seed2029/best.pth.tar
CPU times: user 19min 46s, sys: 1min 27s, total: 21min 14s
Wall time: 4min 56s


In [13]:
y_valid.shape, preds_valid.shape

((832, 4), (832, 4))

In [14]:
from sklearn.metrics import roc_auc_score, confusion_matrix
import copy

In [15]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def cal_metric(logit, truth):
    #pred = sigmoid(logit.cpu().detach())
    pred = sigmoid(logit)
    #truth = truth.cpu().detach().numpy()
    ##
    THRESHOLD_candidate = np.arange(0.01, 0.99, 0.01)
    N = len(THRESHOLD_candidate)
    best_threshold = [0.5, 0.5, 0.5, 0.5]
    best_score = -1
    tn, fp, fn, tp, pos_percent = 0, 0, 0, 0, 0.0

    for ch in range(4):
        for i in range(N):
            THRESHOLD = copy.deepcopy(best_threshold)
            THRESHOLD[ch] = THRESHOLD_candidate[i]
            _pred = pred>THRESHOLD
            _pred, truth = _pred.reshape(-1, 1), truth.reshape(-1, 1)
            
            _tn, _fp, _fn, _tp = confusion_matrix(truth, _pred).ravel()
            _auc = round(roc_auc_score(truth, _pred), 5)
            if _tn+_fn==0:
                fn_rate = 9999
            else:
                fn_rate = round(_fn/(_tn+_fn), 5)
            _pos_percent = (_tp+_fp)/(_tp+_fp+_tn+_fn)
            
            if _auc > best_score:
                best_threshold = copy.deepcopy(THRESHOLD)
                best_score = _auc
                tn, fp, fn, tp, pos_percent = _tn, _fp, _fn, _tp, _pos_percent
    return best_threshold, best_score, tn, fp, fn, tp, pos_percent

In [16]:
best_threshold, best_score, tn, fp, fn, tp, pos_percent = cal_metric(preds_valid, y_valid)
best_threshold, best_score, tn, fp, fn, tp, pos_percent

([0.49, 0.49, 0.49, 0.49], 0.80888, 1255, 297, 339, 1437, 0.5210336538461539)

In [17]:
print(classification_report(y_valid, sigmoid(preds_valid)>best_threshold))

              precision    recall  f1-score   support

           0       0.80      0.74      0.77       424
           1       0.84      0.87      0.86       383
           2       0.78      0.74      0.76       399
           3       0.86      0.86      0.86       570

   micro avg       0.83      0.81      0.82      1776
   macro avg       0.82      0.81      0.81      1776
weighted avg       0.83      0.81      0.82      1776
 samples avg       0.85      0.84      0.81      1776



/home/endi.niu/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [11]:
print(classification_report(y_valid, sigmoid(preds_valid)>best_threshold))

              precision    recall  f1-score   support

           0       0.80      0.62      0.70       434
           1       0.84      0.79      0.82       359
           2       0.82      0.57      0.67       424
           3       0.81      0.84      0.82       559

   micro avg       0.82      0.71      0.76      1776
   macro avg       0.82      0.71      0.75      1776
weighted avg       0.82      0.71      0.76      1776
 samples avg       0.82      0.74      0.74      1776



/home/endi.niu/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [18]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def predict_mask(logit, best_threshold):
    """Transform each prediction into mask.
    input shape: (256, 256)
    """
    #pred mask 0-1 pixel-wise
    #n = logit.shape[0]
    #IMG_SIZE = logit.shape[-1] #256
    #EMPTY_THRESHOLD = 100.0*(IMG_SIZE/128.0)**2 #count of predicted mask pixles<threshold, predict as empty mask image
    #MASK_THRESHOLD = 0.22
    #logit = torch.sigmoid(torch.from_numpy(logit)).view(n, -1)
    #pred = (logit>MASK_THRESHOLD).long()
    #pred[pred.sum(dim=1) < EMPTY_THRESHOLD, ] = 0 #bug here, found it, the bug is input shape is (256, 256) not (16,256,256)
    logit = sigmoid(logit)#.reshape(n, -1)
    pred = (logit>best_threshold).astype(np.int)
    return pred

## predict the testset

In [19]:
import glob
from dataset.dataset import prepare_testset

In [20]:
test_fnames = [f.split('/')[-1][:-4] for f in glob.glob('../data/raw/test/*')]
len(test_fnames), test_fnames[0]

(3698, '10055ff')

In [21]:
test_dl = prepare_testset(BATCH_SIZE, NUM_WORKERS, IMG_SIZE)

In [22]:
%%time

if ensemble:
    preds_test = None
    for checkpoint_path in checkpoint_path_list:
        net = EfficientNet.from_name(MODEL, debug=debug)#override_params={'num_classes': 1}
        in_features = net._fc.in_features
        net._fc = nn.Linear(in_features, 4)#num_classes=1
        net = net.cuda(device=device)
        
        net, _ = load_checkpoint(checkpoint_path, net)
        
        if multi_gpu is not None:
            net = nn.DataParallel(net, device_ids=multi_gpu)
        
        _preds_test = predict_proba(net, test_dl, device, multi_gpu=multi_gpu, mode='test', tta=True)
        
        if preds_test is None:
            preds_test = _preds_test
        else:
            preds_test += _preds_test
        preds_test /= len(checkpoint_path_list)

else:
    preds_test = predict_proba(net, test_dl, device, multi_gpu=multi_gpu, mode='test', tta=True)

use TTA
use TTA
use TTA
use TTA
use TTA
CPU times: user 1h 24min 29s, sys: 5min 21s, total: 1h 29min 51s
Wall time: 16min 24s


In [23]:
preds_test.shape

(3698, 4)

## build submission (for empty/non-empty)

In [96]:
best_threshold = [0.51, 0.51, 0.51, 0.51]

In [97]:
preds_clf = (sigmoid(preds_test)-best_threshold).astype(np.float).reshape(1,-1)[0]
preds_clf

array([-0.08883731, -0.16244433,  0.11221843, ...,  0.0632857 ,
       -0.04195655, -0.0703424 ])

In [98]:
img_id_ch = []
for fname in test_fnames:
    for name in ['Fish', 'Flower', 'Gravel', 'Sugar']:
        img_id_ch.append(fname+'.jpg_%s'%name)

sub_df = pd.DataFrame({'Image_Label': img_id_ch, 'has_mask': preds_clf})
print(len(sub_df.index))

14792


In [99]:
sub_df.head(3)

,Image_Label,has_mask
0,10055ff.jpg_Fish,-0.088837
1,10055ff.jpg_Flower,-0.162444
2,10055ff.jpg_Gravel,0.112218


### use clf on segment model output

In [100]:
seg_df = pd.read_csv('../submission/1110_deeplabv3plus_resnet_512x768_v6_seed2001.csv.gz').fillna('')
seg_df.head(5)

,Image_Label,EncodedPixels
0,10055ff.jpg_Fish,
1,10055ff.jpg_Flower,
2,10055ff.jpg_Gravel,1217 12 1560 2 1564 20 1599 2 1618 4 1899 3 19...
3,10055ff.jpg_Sugar,23 100 266 37 311 5 360 127 592 81 706 135 939...
4,6e54759.jpg_Fish,1829 6 2178 7 2527 8 2876 10 2900 1 3226 10 32...


In [101]:
(seg_df['EncodedPixels']!='').mean()

0.6094510546241212

In [102]:
seg_df = seg_df.merge(sub_df, on=['Image_Label'])

In [103]:
for idx in seg_df.index:
    has_mask = seg_df.loc[idx, 'has_mask']>0
    if not has_mask:
        seg_df.loc[idx, 'EncodedPixels'] = ''

In [104]:
(seg_df['EncodedPixels']!='').mean()

0.4242157923201731

In [105]:
seg_df = seg_df[['Image_Label', 'EncodedPixels']]

In [106]:
seg_df.to_csv('../submission/1110_deeplabv3plus_resnet_512x768_v6_seed2001_with_clf.csv.gz', 
              index=False, compression='gzip')

In [55]:
seg_df.head(5)

,Image_Label,EncodedPixels
0,10055ff.jpg_Fish,
1,10055ff.jpg_Flower,
2,10055ff.jpg_Gravel,1217 12 1560 2 1564 20 1599 2 1618 4 1899 3 19...
3,10055ff.jpg_Sugar,23 100 266 37 311 5 360 127 592 81 706 135 939...
4,6e54759.jpg_Fish,


## or select predicted empty with highest confidence, then remove them

In [21]:
sub_df.sort_values(['has_mask'], ascending=True, inplace=True)

In [128]:
cnt = 5000

candidates = sub_df.iloc[:cnt, ]
print(candidates.has_mask.max())
candidates = candidates.Image_Label.to_list()
candidates[:5]

-0.24907691001892096


['efe623a.jpg_Fish',
 '8ab86a9.jpg_Fish',
 '9c0203c.jpg_Fish',
 '7130cc6.jpg_Fish',
 '971d047.jpg_Fish']

In [28]:
seg_df = pd.read_csv('../submission/1110_deeplabv3plus_resnet_512x768_v11_seed2030.csv.gz').fillna('')
seg_df['has_mask'] = seg_df.EncodedPixels!=''
seg_df.head(5)

,Image_Label,EncodedPixels,has_mask
0,10055ff.jpg_Fish,,False
1,10055ff.jpg_Flower,,False
2,10055ff.jpg_Gravel,3664 10 4005 20 4354 22 4704 23 5054 27 5404 2...,True
3,10055ff.jpg_Sugar,1090 36 1139 1 1438 41 1486 9 1783 63 2132 65 ...,True
4,6e54759.jpg_Fish,,False


In [130]:
seg_df.loc[seg_df.Image_Label.isin(candidates), ].groupby(['has_mask']).agg(['count'])

,Image_Label,EncodedPixels
,count,count
has_mask,,
False,4937,4937
True,63,63


## be careful, now remove masks!

In [131]:
to_remove = seg_df.loc[seg_df.Image_Label.isin(candidates), ].groupby(['has_mask']).get_group(1)['Image_Label'].to_list()
len(to_remove)

63

In [132]:
(seg_df['EncodedPixels']!='').mean()

0.41752298539751215

In [133]:
seg_df.loc[seg_df.Image_Label.isin(to_remove), 'EncodedPixels'] = ''

In [134]:
(seg_df['EncodedPixels']!='').mean()

0.41326392644672794

In [135]:
seg_df = seg_df[['Image_Label', 'EncodedPixels']]

In [111]:
seg_df.to_csv('../submission/1108_deeplabv3plus_resnet_512x768_v6_seed2009_with_clf.csv.gz', index=False, compression='gzip')


**nonzero_classifier_efficientnet-b5_512x768_v2_seed2018.log**
- remove 30, LB.663 to .664
- remove 80, 